In [12]:
import pandas as pd

In [13]:
brand_col = "品牌 Brand"

In [14]:
csv_file = "./processed.csv"
df = pd.read_csv(csv_file)
df = df.loc[:, ~df.columns.str.contains("^Unnamed")]
df = df.drop(df.index[df.index % 10 == 0])
print(df)

brand_list = list(pd.read_csv("brand_list.csv")["brand"])
print(brand_list)

           照片 Picture 品牌 Brand 衣服長度 Clothes Length  價格 Price  毛衣、針織衣 Sweater  \
1     image_00001.jpg      H&M              長 long       999             0.0   
2     image_00002.jpg      H&M              長 long       699             0.0   
3     image_00003.jpg      H&M              長 long       999             0.0   
4     image_00004.jpg      H&M              長 long       699             0.0   
5     image_00005.jpg      H&M             短 short       499             0.0   
...               ...      ...                 ...       ...             ...   
3589  image_03589.jpg    Lativ             短 short       499             0.0   
3591  image_03591.jpg    Lativ             短 short       499             0.0   
3592  image_03592.jpg    Lativ             短 short       399             0.0   
3593  image_03593.jpg    Lativ             短 short       499             0.0   
3594  image_03594.jpg    Lativ             短 short       299             0.0   

      緊身褲 leggings  牛仔褲/裙 jeans / demi 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

df["brand_number"] = df[brand_col].apply(
    lambda x: brand_list.index(x) if x in brand_list else -1
)
for i in range(len(brand_list)):
    df[f"品牌 Brand_{i}"] = (df["brand_number"] == i).astype(int)

df["衣服長度 Clothes Length"] = (df["衣服長度 Clothes Length"] == "長 long").astype(int)
df = df[sorted(df.columns)]


target = "價格 Price"
y = df[target]
X = df.drop(
    columns=[
        target,
        "照片 Picture",
        "品牌 Brand",
        "衣服長度 Clothes Length",
        "brand_number",
    ]
)


print(X)

      毛衣、針織衣 Sweater  緊身褲 leggings  牛仔褲/裙 jeans / demi skirt  下半身 Bottom  \
1                0.0           0.0                       0.0         0.0   
2                0.0           0.0                       0.0         0.0   
3                0.0           0.0                       0.0         1.0   
4                0.0           0.0                       0.0         1.0   
5                0.0           0.0                       0.0         1.0   
...              ...           ...                       ...         ...   
3589             0.0           0.0                       1.0         1.0   
3591             0.0           0.0                       1.0         1.0   
3592             0.0           0.0                       1.0         1.0   
3593             0.0           0.0                       1.0         1.0   
3594             0.0           0.0                       1.0         1.0   

      西裝 Suit  上半身 Top  外套 Jacket  運動材質 Sport Tops  洋裝 Dress  \
1         0.0      1.0 

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [17]:
from sklearn.ensemble import RandomForestRegressor

rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=13)
rf_regressor.fit(X_train, y_train)

RandomForestRegressor(max_depth=13, random_state=42)

In [18]:
y_pred = rf_regressor.predict(X_test)

In [19]:
from sklearn.metrics import (
    mean_absolute_error,
    mean_squared_error,
    r2_score,
    mean_absolute_percentage_error,
)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("mae:", mae)
print("mse:", mse)
print("mape:", mape)
print("r2:", r2)

mae: 426.80973151379106
mse: 621749.5407340876
mape: 0.2924011964828924
r2: 0.761672131365651


In [20]:
import pickle

# Save the model using pickle
with open("./models/random_forest_model.pkl", "wb") as f:
    pickle.dump(rf_regressor, f)

# Alternatively, use joblib
from joblib import dump

dump(rf_regressor, "./models/random_forest_model.joblib")

['./models/random_forest_model.joblib']